In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name = fn, length = len(uploaded[fn])))

In [0]:
!ls

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import time
import os 
import psutil

pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

# DATA PREPROCESSING------------------------------------------------------------
data = pd.read_csv("DARPA1999_week1mon", delimiter=',', header=0, quotechar='"')
tcp_data = data.loc[data['Protocol']=='TCP']

# Gets the next input sequence of packets
def get_sequence(pos, input_data):
    t1 = input_data[pos][1]  # Timestamp of the first packet in the sequence
    src1 = input_data[pos][2]  # Source IP of the first packet in the sequence
    num_packets = 0  # Store the number of packets in the sequence
    sequence = []

    for x in range(pos, pos + 320):
        # if packet does not have the same source or destination IP as the source of the first one in the sequence
        if (input_data[x][2] != src1 and input_data[x][3] != src1 and len(sequence)>3):
            break
        # if the current one did not happen within 1 second of the first one in the sequence
        elif ((input_data[x][1]-t1)>1 and len(sequence)>3):
            break
        else:
            row = input_data[x]
            sequence.append(row)  # Append the sequence with the current packet
            num_packets = num_packets + 1  # Update the number of packets

    return num_packets, sequence

In [0]:
# Split into train, test data
train, test = train_test_split(tcp_data, test_size=0.33, random_state=0, shuffle=False)
print("Train: \n", train)
np_data = np.array(test)

# Get all of the sequences
count = 1  # Store the total number of processed packets

num_sequences = 0  # Store the total number of sequences

all_sequences_df = pd.DataFrame(
    columns=["No.", "Time", "Source IP", "Destination IP", "Protocol", "Length", "Source Port", "Destination Port"])

start_time = time.time()

while (count < len(np_data) and len(np_data)-count>3):
    num_packets, sequence = get_sequence(count, np_data)  # Get the next sequence
    count = count + num_packets  # Update the number of processed packets
    print("Count: ", count)
    print("np_data: ", len(np_data))

    sequence_df = pd.DataFrame(data=sequence, columns = ["No.", "Time", "Source IP", "Destination IP", "Protocol", "Length", "Source Port", "Destination Port"])
    print('Number of packets in sequence: ', num_packets)
    print("Sequence: \n", sequence_df)
    all_sequences_df = all_sequences_df.append(sequence_df)

    num_sequences = num_sequences + 1  # Update the number of sequences
    print("Sequence Number: ", num_sequences)
    print("=========================================================\n")

print("\n\n\n@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
print("-----> FINISHED READING SEQUENCES <-----")
print("Time of Execution: %s seconds" % (time.time() - start_time))

In [0]:
#print(all_sequences_df)

#convert dataframe to csv file 
#sequenceCSV = all_sequences_df.to_csv("/content/SequenceCSV.csv")

import psutil 

print(psutil.virtual_memory())

print(psutil.virtual_memory()[0])
print(psutil.virtual_memory()[1])
print(psutil.virtual_memory()[2])

print("Total: ", psutil.virtual_memory()[0] / 1024/1024/1024)